In [90]:
import pandas as pd
import os
from datetime import datetime

In [91]:
#importing the extracted files for transformation 
root_extract_dir = '../extract'
customers_data = pd.read_csv(f'{root_extract_dir}/customers_large.csv')
orders_data = pd.read_csv(f'{root_extract_dir}/orders_large.csv')
returns_data = pd.read_csv(f'{root_extract_dir}/returns_large.csv')

In [92]:
#converting the data to pd dataframe
customer_df = pd.DataFrame(customers_data)
orders_df = pd.DataFrame(orders_data)
returns_df = pd.DataFrame(returns_data)

In [93]:
rows_in = {'customers':len(customer_df),'orders':len(orders_df),'returns':len(returns_df)}
rows_out = {'customers':0,'orders':0,'returns':0}
nulls_dropped = {'customers':0,'orders':0,'returns':0}
invalid_types_handled = {'customers':0,'orders':0,'returns':0}
processed_time = datetime.now()
summary={'customers':{},'orders':{},'returns':{}}

In [94]:
#customers dataframe
#Trim whitespace in name
customer_df['name'] = customer_df['name'].str.strip()
#Convert signup_date to datetime (handle format inconsistencies)
customer_df['signup_date'] = pd.to_datetime(customer_df['signup_date'], errors='coerce')
count_of_invalid_customer_date = customer_df['signup_date'].isna().sum()
#Drop rows with missing customer_id or name
missing_id_count = customer_df['customer_id'].isna().sum()
missing_name_count = customer_df['name'].isna().sum()
total_null_count = missing_id_count + missing_name_count
summary['customers']['missing_id_name_count']=total_null_count
summary['customers']['count_of_invalid_customer_date']=count_of_invalid_customer_date

customer_df.dropna(subset=['name','customer_id'], inplace=True)
customer_df.drop_duplicates(inplace=True)

rows_out['customers'] = len(customer_df)




In [95]:
#orders dataframe
#Convert order_date to datetime (log and drop invalid)
orders_df['order_date'] = pd.to_datetime(orders_df['order_date'], errors='coerce')
count_of_invalid_order_date = orders_df['order_date'].isna().sum()
orders_df.dropna(subset=['order_date'], inplace=True)
#Convert amount to float (handle missing or malformed values)
orders_df['amount'] = pd.to_numeric(orders_df['amount'], errors='coerce')
count_of_invalid_order_amount = orders_df['amount'].isna().sum()
summary['orders']['count_of_invalid_order_amount']=count_of_invalid_order_amount
summary['orders']['count_of_invalid_order_date']=count_of_invalid_order_date
orders_df.dropna(subset=['amount'], inplace=True)

rows_out['orders']=len(orders_df)




In [98]:
print(rows_out)
print(summary)

{'customers': 990, 'orders': 4848, 'returns': 0}
{'customers': {'missing_id_name_count': np.int64(10), 'count_of_invalid_customer_date': np.int64(5)}, 'orders': {'count_of_invalid_order_amount': np.int64(43), 'count_of_invalid_order_date': np.int64(109)}, 'returns': {}}
